In [110]:
%%time
#%%writefile RDkit.py

""" Here are the functions: 
    to calculate similarity between: 
        two fasta sequences (using BioPython)
        one fasta and the whole database targets (using BioPython)
        SMILES and list of SMILES (using RDkit)
        SMILES and the whole database ligands (using RDkit)
        ligand and the whole database (using pybel fingerprints)
        two protein structure files (TM-score and RMSD, using TM-align) # in process

RDkit: https://github.com/rdkit/rdkit/blob/master/Docs/Book/Install.md
TM-align: https://zhanglab.ccmb.med.umich.edu/TM-align/
Open Babel: http://openbabel.org/docs/current/UseTheLibrary/Python_Pybel.html
Biopython: https://biopython.org/
"""

import os
import pandas as pd
import subprocess
import json
from pathlib import Path
import ntpath

# More about RDkit https://www.rdkit.org/docs/Cookbook.html
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols

import openbabel
import pybel

from Bio import pairwise2
import Bio.SubsMat.MatrixInfo
from Bio.SubsMat.MatrixInfo import *

import DATABASES_SMILES as db
#import Drugbank as dr

from Bio import SeqIO  # Doesn't work

##################   SEQUENCE SIMILARITY (for targets)   ####################################

# Useful if Bio.SeqIO doesn't work somehow, otherwise useless    
class get_seq_from_fasta_file:
    """ Get name (first line of .fasta) and string from .fasta (Biopython's SeqIO sometimes doesn't work:( ))"""
    def __init__(self, path):
        with open(path, 'r') as f:
            seq = ''
            for line in f.readlines():
                if line[0] == '>':
                    name = line.rstrip()
                else:
                    seq += line.rstrip()
            self.name = name
            self.seq = seq


def get_sequences_similarity(input1, input2, align_matrix='blosum62', verbose=False):
    """ Calculates similarity of two sequences using align_matrix from Biopython (blosum62 by default) and 
    paths to fastas or sequences
    Input -  sequences or paths to single fastas
    Output - integers similiarity and identity
    """
    # Draft for using different substitution matrices
    #print('Available matrices:', Bio.SubsMat.MatrixInfo.available_matrices)
    #print('Which one would you like to use? Type [Enter] to use blosum62')
    #align_matr = input()
    # Process input data
    # If first input is path to fasta
    if input1.split('.')[-1] == 'fasta':
        # If import from Bio import SeqIO works
        seq1 = Bio.SeqIO.read(input1, "fasta")
        # If Bio.SeqIO doesn't work
        #seq1 = get_seq_from_fasta_file(input1)
        str1 = seq1.seq        
    # If first input is string
    else:
        str1 = input1
     # If second input is path to fasta    
    if input2.split('.')[-1] == 'fasta':
        # If import from Bio import SeqIO works
        seq2 = Bio.SeqIO.read(input2, "fasta")
        # If Bio.SeqIO doesn't work
        #seq2 = get_seq_from_fasta_file(input2)     
        str2 = seq2.seq
    # If second input is string
    else:
        str2 = input2
    # Get needed matrix
    exec('matr_bio = Bio.SubsMat.MatrixInfo.' + align_matrix, globals())
    # Make an alignment
    try:
        alignments = pairwise2.align.globalds(str1, str2, matr_bio, -10, -0.5)  
        alignments_id = pairwise2.align.globalms(str1, str2, 1, 0, 0, 0) 
        # Print info
        if verbose:
            print("Matrix " + align_matrix + ", number of alignments = " + str(len(alignments)))
            print(pairwise2.format_alignment(*alignments[0]))
        sim = int(alignments[0][2])
        ident = int(str(alignments_id[0][2]).split('.')[0])
        return sim, ident
    except:
        print('Wasn\'t able to compare with ', input2)
        return -1000, 0


def get_closest_fastas_in_fasta_file_from_fasta_or_seq(fasta, path_to_data_in_fasta, 
                                                       k=0, align_matrix='blosum62', sim_or_ident=True):
    """ Returns k of closest to input fasta molecules from path_to_data_in_fasta multi-fasta.
    OUTPUT -- dataframe: 'query':repeated input fasta, 
                    'position_in_fasta': position in input file (to find later needed info)
                     'similarity':similarity, 'identity': identity, 
                     'sequence': sequence of compared target, 'name':name of compared target
            Also writes the best alignment
    INPUT -- fasta -- input a/a sequence of path to single fasta file,
            path_to_data_in_fasta -- fasta file to compare with, 
            k -- number of the best to find (k == 0 if want to get all), 
            sim_or_ident == True => get closest by similiarity, othrwise by identity
    """
    # Load fastas to compare with
    records = list(SeqIO.parse(path_to_data_in_fasta, "fasta"))
    # Process when input is path to fasta file
    if fasta.split('.')[-1] == '.fasta':
        seq = get_seq_from_fasta_file(fasta).seq
    # If input is sequence string
    else:
        seq = fasta
    # Get similarities and identities for all targets in Drugbank
    similarity_list = [] 
    identity_list = []
    seq_list = []
    name_list = []
    for ind, element in enumerate(records):
        if element.seq == seq and k == 1:
            sim, ident = get_sequences_similarity(seq, element.seq)
            d =  {'query':fasta, 'position_in_fasta':ind,  
                  'similarity':sim, 'identity':ident,
                  'sequence':seq, 'name':element.name,
                 }
            return pd.DataFrame(data=d)
        sim, ident = get_sequences_similarity(seq, element.seq, align_matrix)
        similarity_list.append(sim)
        identity_list.append(ident)
        seq_list.append(element.seq)
        name_list.append(element.name)
    # Create correspondent dictionary and then dataframe
    d = {'query':[fasta]*len(similarity_list), 'position_in_fasta':range(len(similarity_list)), 
         'similarity':similarity_list, 'identity':identity_list,
         'sequence':seq_list, 'name':name_list,
        }
    res = pd.DataFrame(data=d)
    if sim_or_ident:
        res = res.sort_values('similarity', ascending=False)
    else:
        res = res.sort_values('identity', ascending=False)
    if k < res.size and k != 0:
        result = res[0:k]
    else:
        result = res
    get_sequences_similarity(seq, get_element_of_fasta_by_number(path_to_data_in_fasta, 
                                                                 result.iloc[0].loc['position_in_fasta']),
                                                                verbose=True)
    return result
                   

def get_closest_fastas_from_uniprot(uniprot, path_to_data_in_fasta, k=0, sim_or_ident=True):
    """ Returns k of closest to fasta of input molecule (with uniprot Uniprot ID) molecules 
    from path_to_data_in_fasta multi-fasta.
    OUTPUT -- 'query':repeated input fasta, 
                    'position_in_fasta': position in input file (to find later needed info)
                     'similarity':similarity, 'identity': identity, 
                     'sequence': sequence of compared target, 'name':name of compared target
    INPUT -- path to fasta or sequence, k -- number of the best to find (k == 0 if want to get all), 
            sim_or_ident == True => get closest by similiarity, othrwise by identity
    """
    seq = db.get_seq_from_uniprot(uniprot)
    return get_closest_fastas_in_fasta_file_from_fasta_or_seq(seq, path_to_data_in_fasta, k, sim_or_ident)

                   
def get_element_of_fasta_by_number(path_to_data_in_fasta, n):
    """ Returns SeqIO record of n-th order from multi-fasta file.
    OUTPUT -- SeqIO fasta sequence element
    INPUT -- path to fasta file with compared fastas, number of needed element in this fasta."""
    # Load fastas with thich compared           
    records = list(SeqIO.parse(path_to_data_in_fasta, "fasta"))
    return records[n]                           


def print_closest_fastas(path_to_data_in_fasta, list_of_numbers):
    """ Get name and sequence of elements with numbers in 'list_of_numbers' from 'path_to_data_in_fasta'"""
    records = list(SeqIO.parse(path_to_data_in_fasta, "fasta"))
    for n in list_of_numbers:
        elem = records[n]
        print('Name = ', elem.name)
        print('Seq = ', elem.seq)


##################   SMILES SIMILARITY (for ligands)   ####################################

def get_smiles_similiarity(smiles, list_smiles):
    """ Get dataframe 'query':input SMILES, 
                    'target_smiles':list_smiles_cleaned, 
                    'similarity':similarities correspondently
    INPUT -- SMILES (smiles) and list of smiles to compare with (list_smiles)
    """
    # Proof and make a list of SMILES
    c_smiles = []
    # Delete Nones
    list_smiles_cleaned = [i for i in list_smiles if i]
    # List of indices to delete because SMILES are invalid
    del_indices = []
    for ind, ds in enumerate(list_smiles_cleaned):
        try:
            cs = Chem.CanonSmiles(ds)
            c_smiles.append(cs)
        except:
            # Delete smiles if it's invalid
            del_indices.append(ind)
            print('Invalid SMILES, deleted from list:', ds)
    # Delete elements starting from end
    for ind in del_indices[::-1]:
        del list_smiles_cleaned[ind]
    try:
        smiles = Chem.CanonSmiles(smiles)
    except:
        print('Invalid Input SMILES:', ds)
        return -1

    # Make a list of mols
    ms = [Chem.MolFromSmiles(x) for x in c_smiles]

    # Make a list of fingerprints (fp)
    fps = [FingerprintMols.FingerprintMol(x) for x in ms]
    # Input fingerprint
    fp_in = FingerprintMols.FingerprintMol(Chem.MolFromSmiles(smiles))

    # Compare all fps with fp_in
    sim = (DataStructs.BulkTanimotoSimilarity(fp_in, fps[:]))
    #print()

    # Build the dataframe and sort it
    print(len([smiles]*len(sim)), len(list_smiles_cleaned), len(sim))
    d = {'query':[smiles]*len(sim), 'smiles':list_smiles_cleaned, 'similarity':sim}
    df_final = pd.DataFrame(data=d)
    df_final = df_final.sort_values('similarity', ascending=False)
    return df_final#dict(zip(df_final['Similarity'], df_final['target']))


def get_closest_smiles_names(smiles, k=1):
    """ Get k names and smiles of the closest to input smiles, k=1 by default
    INPUT -- SMILES (smiles), k -- number of the best smiles to find (k == 0 if want to get all)
    OUTPUT -- dataframe of similar by smiles ligands: 
            'name' -- names of ligands
            'smiles' -- SMILES of ligands
            'query' -- input SMILES (same for all)
            'similarity' -- level of similarity (1 - identical, 0 - abs. different)
    """
    # Load needed dictionary
    global root
    db.load_info_db_from_namelist(['ligands_names_and_smiles'], root)
    # Delete ligands with None smiles
    dict_cleaned = {k: v for k, v in ligands_names_and_smiles.items() if v is not None}
    # Get dataframe of sorted by descending similarity smiles
    try:
        res = get_smiles_similiarity(smiles, list(dict_cleaned.values()))
    except:
        if res == -1:
            print('Input SMILES is invalid, abort')
            return -1
    # Take only needed amount of smiles
    if k < res.size and k != 0:
        result = res[0:k]
    else:
        result = res
    # Get names of correspondent ligands
    names = []
    for sm in result['smiles']:
        for name in ligands_names_and_smiles.keys():
            if ligands_names_and_smiles[name] == sm:
                names.append(name)
    result['name'] = names
    return result


##################   Structure-Fingerprints SIMILARITY (for ligands)   ####################################

def change_extension(path, new_ext):
    """ Returns path with changed extension
    INPUT - path and new extension
    OUTPUT -- path with changed to new_ext extension, if path had extension. If not returns -1"""
    f, f_ext = os.path.splitext(path)
    if f_ext == '':
        print('Path without extension')
        return -1
    return f + new_ext


def get_TMscore_and_RMSD(struct1_path, struct2_path):
    """ Get TM-score and RMSD of two protein structure files
    INPUT -- paths of two protein structure files (.sdf, mol2, pdb)
    OUTPUT -- (TM-score, RMSD) of the files
        if 0.0 < TM-score < 0.17, then random structural similarity 
        if 0.5 < TM-score < 1.00, then in about the same fold 
        Returns 0.0 if no common residues were found
    """
    # Convert files to .pdb if needed, saving in the same directory and changing extension
    pdb1_path = change_extension(struct1_path, '.pdb')
    pdb2_path = change_extension(struct2_path, '.pdb')
    db.convert_single_structure(struct1_path, pdb1_path)
    db.convert_single_structure(struct1_path, pdb2_path)
    # ?? Delete new structures after calculation or not??
    res = subprocess.check_output(['TMscore', pdb1_path, pdb2_path])
    text = res.decode('utf-8')
    #print(text)
    if text.find('TM-score') == -1:
        return(0.0)
    else:
        tm_score = float(text.split('TM-score')[3].split()[1])
        rmsd = float(text.split('RMSD')[1].split()[4])
        return tm_score, rmsd


def extract_approved_sdf(path_to_sdf_from_drugbank, root, overwrite=False):
    """ Extract approved ligands taken from 'ligands_drugbank_ids' to new multi-sdf file
    with changed name as added by _approved. Overwrite - flag of overwriting this file
    """
    # Set path of file with approved sdfs
    path_to_approved_sdf = path_to_sdf_from_drugbank.split('.sdf')[0] + '_approved.sdf'
    # If file with approved structured doesn't exist or should be overwrited
    if not Path(path_to_approved_sdf).is_file() or overwrite:
        # Load list of ids of approved ligands
        load_info_db_from_namelist(['ligands_drugbank_ids'], root)
        sdf_approved = pybel.Outputfile("sdf", path_to_approved_sdf, overwrite=True)
        for mol in pybel.readfile('sdf', path_to_sdf_from_drugbank):
            mol_id = mol.data['DATABASE_ID']
            # Check if ligand is in approved list
            f_approved = False
            for lig in ligands_drugbank_ids:
                if mol_id in lig:
                    f_approved = True
                    break
            if f_approved:
                sdf_approved.write(mol)
        sdf_approved.close()
    return path_to_approved_sdf


def get_closest_ligands_from_3d_structure(path_to_structure, path_to_sdf_approved, root, fptype='fp2', number_to_print=1):
    """ Get sorted descending list of tanimoto coeff from fingerprints and correspondent DB IDs list
    More about fingerprints http://openbabel.org/docs/current/UseTheLibrary/Python_Pybel.html
    Their formats: http://openbabel.org/docs/current/Fingerprints/fingerprints.html#fingerprint-format-details
    INPUT:
        path_to_structure -- path to single .sdf, .pdb or .mol2 structure of molecule
        path_to_sdf_approved -- path to multi-sdf file to compare with
        fptype - type of fingerptint, such as 'ftp2', 'maccs', 'ecfp0' etc.
        list of all available can be taken by 'pybel.fps'
        number_to_print - how many to print
    OUTPUT:
        dataframe: 'Name' : name of compared ligand,'Tanimoto coeff' : corresp similarity,
        'Drugbank ID' : Drugbank ID of compared ligand, 'Fingerprint type' : name of used fingerprint      
    """
    # Fingerprints of ligands
    fps = []
    # Drugbank IDs of ligands
    ids = []
    # Tanimoto coefficients between fingerprints
    tanim = []
    for mymol in pybel.readfile('sdf', path_to_structure):
        fp_mol = mymol.calcfp(fptype)
    for mol in pybel.readfile('sdf', path_to_sdf_approved):
        # Get correspondent fingerprint
        fp = mol.calcfp(fptype)
        fps.append(fp)
        tanim.append(fp_mol | fp)
        # Get ID in Drugbank
        ids.append(mol.data['DATABASE_ID'])
    tanim, ids= zip(*sorted(zip(tanim, ids)))
    # Make them descending
    tanim = tanim[::-1]
    ids = ids[::-1]
    # Get names of ligands
    load_info_db_from_namelist(['ligands_names', 'ligands_drugbank_ids'], root)
    ligands_db_ids_by_names = dict(zip(ligands_names, ligands_drugbank_ids))
    names = []
    for lig_id in ids:
        for name in ligands_db_ids_by_names.keys():
            if lig_id in ligands_db_ids_by_names[name]:
                names.append(name)
    # Make dataframe from obtained data
    data_tuples = list(zip(names, tanim, ids, [fptype]*len(ids)))
    df = pd.DataFrame(data_tuples, columns=['Name','Tanimoto coeff', 'Drugbank ID', 'Fingerprint type'])
    # Print the best k
    print(df[0:number_to_print])
    return df

##################   Structure TM-score and RMSD SIMILARITY (for targets)   ####################################
def get_closest_targets_from_3d_structure(path_to_structure, path_to_sdf_from_drugbank, k=0):
    """"""
    # Load file with all sdfs to compare
    sdf_drugbank = pybel.readfile('sdf', path_to_sdf_from_drugbank)
    # New path to converted to pdb structure
    new_path = change_extension(path_to_structure, '.pdb')
    # Convert input structure to PDB
    db.convert_structure(path_to_structure, new_path)
    x.calcfp()
    tm_list = []
    rmsd_list = []
    # Make directory where to save all pdbs
    dir_name = str(Path(ntpath.dirname(path_to_structure)) / 'pdbs_ligands')
    db.make_dir(dir_name)


def load_info_db_from_namelist(namelist, root):
    """Load listed in namelist names.txt collected from Drugbank data as json files from root/Drugbank_exracted"""
    # All names of files to be loaded from root/Drugbank_extracted with name.txt, where name is from names
    name_full = str(Path(root) / 'Drugbank_extracted')
    for name in namelist:
        with open(str(Path(name_full) / (name + ".txt")), 'r') as f:
            exec('global ' + name + '\n' + name + ' = json.load(f)')
# Tests
#print(get_TMscore_and_RMSD('hive/pdb/1A3B.pdb', 'hive/pdb/1RDQ.pdb')) #1MD7 1RDQ
#print(get_sequences_similarity('P08069.fasta', 'P00533.fasta', verbose=True))
#print(get_sequences_similarity('ABB', 'ABC'))
#a = get_SMILES_similiarity('ClCCNC(=O)N(CCCl)N=O', 
#                       ('CN1CCC[C@@H]1CCO[C@](C)(C1=CC=CC=C1)C1=CC=C(Cl)C=C1', 'ClCCNC(=O)N(CCCl)N=O'))
#print(a)

# Test of seq search
#root = '/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR'
root = '/home/anton_maximov/BACHELOR'
uniprot = 'P00533'
#print(db.get_seq_from_uniprot(uniprot))
fasta = '/home/anton_maximov/BACHELOR/P08069.fasta'
path_to_data_in_fasta = '/home/anton_maximov/BACHELOR/Drugbank_extracted/Drugbank_targets.fasta'
#df = get_closest_fastas_in_fasta_file_from_fasta_or_seq(fasta, path_to_data_in_fasta, k=3, sim_or_ident=True)
#print(df['position_in_fasta'], df['similarity'])
#print(df)
path_to_structure = str(Path(root) / 'Drugbank_extracted' / 'SDF_ideal.sdf')
path_to_sdf_from_drugbank =  str(Path(root) / 'Drugbank_extracted' / 'structures.sdf') #str(Path(root) / 'Drugbank_extracted' / 'structures_approved_by_db.sdf')#
#path_to_sdf_approved = extract_approved_sdf(path_to_sdf_from_drugbank, root, overwrite=True)

#get_closest_ligands_from_3d_structure(path_to_structure, path_to_sdf_approved, root,
#                                                          fptype='maccs', number_to_print=5)

              Name  Tanimoto coeff Drugbank ID Fingerprint type
0  Tiludronic acid        0.727273     DB01133            maccs
1   Clodronic acid        0.580645     DB00720            maccs
2  Phosphoric acid        0.535714     DB09394            maccs
3   Oxidronic acid        0.484848     DB14159            maccs
4        Foscarnet        0.484848     DB00529            maccs
CPU times: user 1.99 s, sys: 0 ns, total: 1.99 s
Wall time: 1.99 s


TypeError: get_closest_ligands_from_3d_structure() got an unexpected keyword argument 'number_to_print'

In [68]:
print(len(mol_id_list))
print(len(tanim))
print(len(ligands_drugbank_ids))
print(tanim)

9680
1940
3220
(0.34328358208955223, 0.3176470588235294, 0.3088235294117647, 0.2905982905982906, 0.2682926829268293, 0.26582278481012656, 0.2631578947368421, 0.25, 0.25, 0.24, 0.23655913978494625, 0.23595505617977527, 0.22857142857142856, 0.22580645161290322, 0.2222222222222222, 0.22105263157894736, 0.22105263157894736, 0.2169811320754717, 0.21649484536082475, 0.21138211382113822, 0.21138211382113822, 0.2111111111111111, 0.21052631578947367, 0.208955223880597, 0.2079207920792079, 0.2079207920792079, 0.20754716981132076, 0.20754716981132076, 0.20754716981132076, 0.20388349514563106, 0.2037037037037037, 0.2, 0.19696969696969696, 0.19658119658119658, 0.19658119658119658, 0.19469026548672566, 0.1941747572815534, 0.19327731092436976, 0.1926605504587156, 0.1926605504587156, 0.19130434782608696, 0.19090909090909092, 0.19047619047619047, 0.1891891891891892, 0.18867924528301888, 0.18840579710144928, 0.18803418803418803, 0.18796992481203006, 0.18705035971223022, 0.18518518518518517, 0.1846153846

In [94]:
pybel.fps

['ecfp0',
 'ecfp10',
 'ecfp2',
 'ecfp4',
 'ecfp6',
 'ecfp8',
 'fp2',
 'fp3',
 'fp4',
 'maccs']

In [ ]:
'DB13269', 'DB01133', 'DB11121', 'DB06820', 'DB00629', 'DB00648', 'DB01018', 'DB04871', 'DB00756', 'DB01218', 'DB01156', 'DB00181', 'DB01407', 'DB01178', 'DB00939', 'DB01122', 'DB01105',

In [8]:
def get_element_of_fasta_by_number(path_to_data_in_fasta, n):
    """OUTPUT -- SeqIO fasta sequence element
    INPUT -- path to fasta file with compared fastas, number of needed element"""
    # Load fastas with thich compared           
    records = list(SeqIO.parse(path_to_data_in_fasta, "fasta"))
    return records[n]  

#print(get_element_of_fasta_by_number(path_to_data_in_fasta, 63).seq)
#print()
#print(get_element_of_fasta_by_number(path_to_data_in_fasta, 62).seq)
#print()
#print(Bio.SeqIO.read(fasta, "fasta").seq)
print_closest_fastas('/home/anton_maximov/BACHELOR/Drugbank_extracted/Drugbank_targets.fasta', [0])
#print(df.iloc[0].loc['position_in_fasta'])

Name =  lcl|BSEQ0016004|Prothrombin
Seq =  MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANTFLEEVRKGNLERECVEETCSYEEAFEALESSTATDVFWAKYTACETARTPRDKLAACLEGNCAEGLGTNYRGHVNITRSGIECQLWRSRYPHKPEINSTTHPGADLQENFCRNPDSSTTGPWCYTTDPTVRRQECSIPVCGQDQVTVAMTPRSEGSSVNLSPPLEQCVPDRGQQYQGRLAVTTHGLPCLAWASAQAKALSKHQDFNSAVQLVENFCRNPDGDEEGVWCYVAGKPGDFGYCDLNYCEEAVEEETGDGLDEDSDRAIEGRTATSEYQTFFNPRTFGSGEADCGLRPLFEKKSLEDKTERELLESYIDGRIVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLLYPPWDKNFTENDLLVRIGKHSRTRYERNIEKISMLEKIYIHPRYNWRENLDRDIALMKLKKPVAFSDYIHPVCLPDRETAASLLQAGYKGRVTGWGNLKETWTANVGKGQPSVLQVVNLPIVERPVCKDSTRIRITDNMFCAGYKPDEGKRGDACEGDSGGPFVMKSPFNNRWYQMGIVSWGEGCDRDGKYGFYTHVFRLKKWIQKVIDQFGE


In [10]:
struct1_path = '/home/anton_maximov/BACHELOR/Drugbank_extracted/SDF_ideal.sdf'
struct2_path = '/home/anton_maximov/BACHELOR/Drugbank_extracted/SDF_ideal.pdb'
#db.convert_single_structure(struct1_path, struct2_path)
get_TMscore_and_RMSD(struct1_path, struct2_path)

 There is no common residues in the input structures



0.0

In [34]:
pybel.fps

['ecfp0',
 'ecfp10',
 'ecfp2',
 'ecfp4',
 'ecfp6',
 'ecfp8',
 'fp2',
 'fp3',
 'fp4',
 'maccs']

In [ ]:
https://www.rcsb.org/pdb/download/downloadFastaFiles.do?structureIdList=1IVO&compressionType=uncompressed

In [15]:
pdb_dir = '/home/anton_maximov/BACHELOR/pdbs'
pdb_list = db.get_pdbs_from_uniprot('P00533')
for pdb in pdb_list:
    db.download_pdb(pdb, pdb_dir)
struct1_path = '/home/anton_maximov/BACHELOR/pdbs/' + pdb_list[0] + '.pdb'
for pdb in pdb_list[1:]:
    struct1_path = '/home/anton_maximov/BACHELOR/pdbs/' + pdb + '.pdb'
    print(f'{pdb} {get_TMscore_and_RMSD(struct1_path, struct2_path)}')

1DNR (1.0, 0.0)
1IVO (0.5054, 37.432)
1M14 (1.0, 0.0)
1M17 (1.0, 0.006)
1MOX (0.5017, 37.343)
1NQL (0.9408, 5.826)
1XKK (1.0, 0.0)
1YY9 (0.5928, 51.98)
1Z9I (1.0, 0.0)
2EB2 (1.0, 0.0)
2EB3 (1.0, 0.0)
2EXP (0.5759, 17.215)
2EXQ (0.7328, 23.519)
2GS2 (1.0, 0.0)
2GS6 (1.0, 0.0)
2GS7 (0.5328, 29.906)
2ITN (1.0, 0.01)
2ITO (1.0, 0.0)
2ITP (1.0, 0.002)
2ITQ (1.0, 0.009)
2ITT (1.0, 0.004)
2ITU (1.0, 0.006)
2ITV (1.0, 0.008)
2ITW (1.0, 0.002)
2ITX (1.0, 0.0)
2ITY (1.0, 0.0)
2ITZ (1.0, 0.0)
2J5E (1.0, 0.0)
2J5F (1.0, 0.0)
2J6M (1.0, 0.0)
2JIT (0.5302, 25.941)
2JIU (0.5293, 26.045)
2JIV (0.536, 27.775)
2KS1 (1.0, 0.0)
2M0B (0.5482, 8.478)
2M20 (1.0, 0.0)
2N5S (1.0, 0.0)
2RF9 (0.5358, 35.777)
2RFD (0.5324, 24.959)
2RFE (0.3279, 38.944)
2RGP (1.0, 0.0)
3B2U (0.1789, 67.92)
3B2V (0.7824, 13.137)
3BEL (1.0, 0.0)
3BUO (0.5347, 26.075)
3C09 (0.4056, 42.898)
3G5V (0.5776, 17.267)
3G5Y (0.5747, 17.291)
3GOP (1.0, 0.003)
3GT8 (0.287, 53.554)
3IKA (0.5317, 26.0)
3LZB (0.2998, 38.188)
3NJP (0.5211, 36.494)

In [13]:
#input1 = '/home/anton_maximov/BACHELOR/Drugbank_extracted/uniprot-reviewed%3Ayes+AND+proteome%3Aup000005640.fasta'
input1 = '/home/anton_maximov/BACHELOR/Drugbank_extracted/Drugbank_targets.fasta'
records = list(SeqIO.parse(input1, "fasta"))
print(records.seq)
print(len(records))

AttributeError: 'list' object has no attribute 'seq'

In [14]:
for i in range(1,5):

range(1, 5)

In [4]:
input1 = '/home/anton_maximov/BACHELOR/P00533.fasta'
input2 = '/home/anton_maximov/BACHELOR/P08069.fasta'
get_sequences_similarity(input1, input2)

Using blosum62
Number of alignments = 1000
MRPSGTAGAA------LLALLAALC--PASRALEEKKVCQGTSNKLTQLGT-FEDHFLSLQRMFNNCEVVLGNLEITYVQ-----RNYDLSFLKTIQEVAGYVLI----ALNTVERIPLENLQIIRG-NMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWR---DIVSSDFLSNMSMDFQNHLGS--CQKCDPSCP------------------NGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGP-RESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKK--C----------PRNYVVTDHGSCVRAC-------GADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINA-TNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELD-ILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVV-SLNITSL---GLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTK---IISNRGEN-SCKATGQVCHALCSPEG------CW----GPEPRDCVS---------CRNVSR--GRECVDKCN---------------------LLEG-EP------------REFVENSE----------------------------------CIQCHPECLPQ---AMNITCTGRGPDN--------C----IQCAHYID---------------------GPHCVKTCP------------------------------------AGVMGENNTLV------WKYADAGHV-------------------------------------CHLC-HPNCTYGCTG-----------------PG---LEGCPTNG-----PKIPSIATGMV----

(666, 497)

In [24]:
df = get_closest_smiles_names('ClC1=CC=CC=C1CN1CCCC2=C(C1)C=CS2', 3)
print(df, df['name'], df['smiles'],df['query'], df['similarity'])

NameError: name 'df' is not defined

In [ ]:
print(len(targets_names_and_fastas.keys()))

In [13]:
%%writefile RDkit1.py

from pathlib import Path
from rdkit.Chem import AllChem, rdMolAlign
from rdkit import Chem, DataStructs, RDConfig
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D, Generate
import DATABASES_SMILES as db

pdb1 = '1AZM'
pdb2 = '2KI5'
root = '/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR'
path = str((Path(root) / 'RDkit' / 'pdbs'))
db.make_dir(path)
# acetazolamide
smiles1 = 'CC(=O)NC1=NN=C(S1)S(N)(=O)=O'
db.download_pdb(pdb1, path)
pdb1_path = str(Path(path) / (pdb1 + '.pdb'))
# acyclovir
smiles2 = 'NC1=NC(=O)C2=C(N1)N(COCCO)C=N2'
db.download_pdb(pdb2, path)
pdb2_path = str(Path(path) / (pdb2 + '.pdb'))
# The reference molecule
ref1 = Chem.MolFromSmiles(smiles1)
ref2 = Chem.MolFromSmiles(smiles2)
# The PDB conformations
mol1 = Chem.MolFromPDBFile(pdb1_path)
mol1 = AllChem.AssignBondOrdersFromTemplate(ref1, mol1)
mol2 = Chem.MolFromPDBFile(pdb2_path)
mol2 = AllChem.AssignBondOrdersFromTemplate(ref1, mol2)

# pharmacophore fingerprint
factory = Gobbi_Pharm2D.factory
fp1 = Generate.Gen2DFingerprint(mol1, factory, dMat=Chem.Get3DDistanceMatrix(mol1))
fp2 = Generate.Gen2DFingerprint(mol2, factory, dMat=Chem.Get3DDistanceMatrix(mol2))
# Tanimoto similarity
tani = DataStructs.TanimotoSimilarity(fp1, fp2)
print(tani)
# Align them
#rms = rdMolAlign.AlignMol(mol1, mol2)
#print(rms)
# Align them with OPEN3DAlign
#pyO3A = rdMolAlign.GetO3A(mol1, mol2)
#score = pyO3A.Align()
#print(score)

Overwriting RDkit1.py


In [33]:
#%%writefile RDkit1.py

from rdkit import Chem
from rdkit.Chem import AllChem
mol = Chem.MolFromSmiles('NC(=[NH2+])c1ccc(C[C@@H](NC(=O)CNS(=O)(=O)c2ccc3ccccc3c2)C(=O)N2CCCCC2)cc1')
cids = AllChem.EmbedMultipleConfs(mol, numConfs=50, maxAttempts=1000, pruneRmsThresh=0.1)
print(len(cids))
# align the conformers
rmslist = []
AllChem.AlignMolConformers(mol, RMSlist=rmslist)
print(len(rmslist))
# calculate RMS of confomers 1 and 9 separately
rms = AllChem.GetConformerRMS(mol, 1, 9, prealigned=True)

Overwriting RDkit1.py


In [2]:
#%%writefile Openbabel.py
import pybel
mymol = pybel.readstring("smi", "CCCC")
print(mymol.molwt)

58.1222


In [14]:
#%%writefile Openbabel.py
import openbabel
from pathlib import Path

name = 'A3551'
root = '/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR'
path = str(Path(root) / 'openbabel')
convert_structure('pdb', path, 'sdf', path, name)